#**Install Dependencies**

In [ ]:
# @title
# !pip uninstall bitsandbytes -y
# !pip install bitsandbytes --no-cache-dir


In [ ]:
# @title
!pip install --upgrade bitsandbytes

In [ ]:
# @title
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 773.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
!pip install streamlit pyngrok transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.0 MB/s eta 0:00:00


In [ ]:
# @title
pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 8.0 MB/s eta 0:00:00


#**app.py**

In [ ]:
%%writefile app.py
import streamlit as st
import streamlit.components.v1 as components
import json
import os
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# --- Streamlit Config ---
st.set_page_config(page_title="ECHO: Wellness System", layout="wide")

# --- Load Emotion Detection Model ---
@st.cache_resource
def load_emotion_model():
    return pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=False)

emotion_classifier = load_emotion_model()

# --- Load Chat Model ---
@st.cache_resource
# def load_chat_model():
#     tokenizer = AutoTokenizer.from_pretrained("RGBEKKO/llama-2-7b-modified-emotion-classifier", use_fast=True)
#     model = AutoModelForCausalLM.from_pretrained("RGBEKKO/llama-2-7b-modified-emotion-classifier")
#     return tokenizer, model

# tokenizer, model = load_chat_model()

def load_chat_model():
    model_name = "RGBEKKO/llama-2-7b-modified-emotion-classifier"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        device_map="auto"
    )
    return tokenizer, model

tokenizer, model = load_chat_model()


system_prompt = """
You are a friendly and empathetic AI assistant focused on mental wellness. Keep your responses moderate around 200 words, conversational, and supportive.
Ask gentle follow-up questions to keep the user engaged, and offer encouragement where appropriate. Prioritize clarity, warmth, and emotional sensitivity.
"""

# --- Chat Model Response ---
import re

def chat_response(prompt):
    wrapped_prompt = f"[INST]<<SYS>>\n{system_prompt}<</SYS>>\n\n{prompt}[/INST]"
    device = "cuda" if torch.cuda.is_available() else "cpu"

    response = ""
    word_count = 0

    while word_count < 150:
        inputs = tokenizer(wrapped_prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=300,
                do_sample=True,
                top_p=0.95,
                temperature=0.8,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id,
            )

        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_text = decoded.split("[/INST]")[-1].strip()
        response += " " + generated_text
        word_count = len(response.split())

    # Trim to ~200 words (if desired)
    sentences = re.split(r'(?<=[.!?])\s+', response)
    final_response = []
    count = 0
    for sentence in sentences:
        words_in_sentence = len(sentence.split())
        if count + words_in_sentence > 200:
            break
        final_response.append(sentence)
        count += words_in_sentence

    return " ".join(final_response).strip()


# --- Rule-Based Chat with LLM Fallback ---
def get_chat_response(user_message, emotion):
    # predefined_responses = {
    #     "sadness": "I'm really sorry you're feeling this way. Want to talk about it more?",
    #     "joy": "That's wonderful! Tell me more about what's making you happy.",
    #     "anger": "It’s okay to feel angry. Want to share what caused it?",
    #     "fear": "That sounds scary. I’m here for you. Would you like to reflect on it?",
    #     "love": "Love is powerful. Would you like to share more about it?",
    #     "surprise": "That’s surprising! Do you want to tell me more?"
    # }
    # if emotion in predefined_responses:
    #     return predefined_responses[emotion]
    return chat_response(user_message)


st.markdown("""
    <style>
    body {
        background-color: #F0F2F6;
    }
    .main {
        background: linear-gradient(145deg, #e3f2fd, #ffffff);
        padding: 2rem;
        border-radius: 1rem;
        box-shadow: 0px 0px 15px rgba(0, 0, 0, 0.05);
    }
    </style>
""", unsafe_allow_html=True)

# --- Session Initialization ---
if "page" not in st.session_state: st.session_state.page = "home"
if "chat_history" not in st.session_state: st.session_state.chat_history = []
if "emotion_log" not in st.session_state: st.session_state.emotion_log = []

# --- Emotion Detection ---
def detect_emotion(text):
    result = emotion_classifier(text)[0]
    return result['label'].lower()

# --- Emotion-Based Redirect Suggestion ---
def redirect_suggestion(emotion):
    if st.session_state.emotion_log.count(emotion) >= 2:
        if emotion in ["anger", "sadness"]:
            # st.warning("⚠️ You’ve expressed this emotion multiple times.")
            # if st.button("🎮 Redirect to Games"):
            st.session_state.page = "games"
            st.rerun()
        elif emotion in ["joy", "surprise"]:
            # st.info("🌟 You seem joyful often today!")
            # if st.button("📓 Redirect to Journal"):
            st.session_state.page = "journal"
            st.rerun()

# --- Pages ---
def home():
    st.title("🏠 Welcome to ECHO")
    st.markdown("#### Your personal mental wellness support system 💖")

def chatbot():
    st.title("💬 ECHO: Mental Wellness Chatbot")

    for chat in st.session_state.chat_history:
        st.markdown(f"🧍‍♀️ **You:** {chat['user']}")
        st.markdown(f"🤖 **ECHO:** {chat['bot']}")
        st.markdown(f"🧠 _Emotion: {chat['emotion'].upper()}_")

    user_input = st.text_input("🧍 You:", key=len(st.session_state.chat_history))

    if user_input:
        emotion = detect_emotion(user_input)
        st.session_state.emotion_log.append(emotion)
        st.info(f"🧠 Detected Emotion: **{emotion.upper()}**")

        bot_reply = get_chat_response(user_input, emotion)
        st.session_state.chat_history.append({"user": user_input, "bot": bot_reply, "emotion": emotion})

        redirect_suggestion(emotion)

        if st.button("💬 Continue Chatting"):
            st.rerun()

    col1, col2 = st.columns(2)
    with col1:
        if st.button("🗑️ Clear Chat"):
            st.session_state.chat_history.clear()
            st.session_state.emotion_log.clear()
            st.rerun()
    with col2:
        chat_json = json.dumps(st.session_state.chat_history, indent=4)
        st.download_button("⬇️ Download Chat", chat_json, "chat_history.json", "application/json")


def games():
    st.title("🎮 Mood-Lifting Games")
    st.success("Let’s boost your mood with something fun!")
    tabs = st.tabs(["🎯 Color Game", "🔢 Number Game"])
    with tabs[0]:
        st.subheader("🎯 Mastermind: Color Edition")
        if os.path.exists("mastermind_color_game.html"):
            components.html(open("mastermind_color_game.html").read(), height=900, scrolling=False)
        else:
            st.error("❌ Color game file missing.")
    with tabs[1]:
        st.subheader("🔢 Mastermind: Number Edition")
        if os.path.exists("mastermind_number_game.html"):
            components.html(open("mastermind_number_game.html").read(), height=900, scrolling=False)
        else:
            st.error("❌ Number game file missing.")


from docx import Document
import io

def journal():
    st.title("📓 Daily Journal")
    st.info("Express yourself and reflect on your day 💬")
    entry = st.text_area("Write here...", height=200)

    if entry:
        doc = Document()
        doc.add_heading("Daily Journal Entry", level=1)
        doc.add_paragraph(entry)

        buffer = io.BytesIO()
        doc.save(buffer)
        buffer.seek(0)

        st.download_button(
            label="💾 Save Journal",
            data=buffer,
            file_name="journal_entry.docx",
            mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document"
        )


# --- Sidebar Navigation ---
with st.sidebar:
    st.header("🧠 ECHO Menu")
    if st.button("🏠 Home"): st.session_state.page = "home"
    if st.button("💬 Chatbot"): st.session_state.page = "chat"
    if st.button("🎮 Games"): st.session_state.page = "games"
    if st.button("📓 Journal"): st.session_state.page = "journal"

# --- Page Routing ---
if st.session_state.page == "home":
    home()
elif st.session_state.page == "chat":
    chatbot()
elif st.session_state.page == "games":
    games()
elif st.session_state.page == "journal":
    journal()


Overwriting app.py


#**Create ngrok account and paste auth token below**

In [ ]:
!ngrok authtoken 2uxNOY5tDpNYbyH06MoLG9UYIwz_4aekW7f1hLsR5JUH5j8Vh

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


#**Run Streamlit App**

In [ ]:
# Run Streamlit App in Notebook
import os
import time
from pyngrok import ngrok

PORT = 8501
os.system(f"streamlit run app.py --server.port {PORT} &")
time.sleep(5)

# Start ngrok tunnel
public_url = ngrok.connect(PORT)
print("Access your chatbot here:", public_url)

Access your chatbot here: NgrokTunnel: "https://8954-34-125-31-228.ngrok-free.app" -> "http://localhost:8501"


# **Kill NGROK Terminal**

In [ ]:
!pkill -f ngrok  #kills existing ngrok tunnel usage(limit 3)

#**GPU Usage**

In [ ]:
import torch
print(torch.cuda.is_available())  # Should print True if using GPU


True


In [ ]:
!nvidia-smi  # For GPU status
!free -h     # For RAM usage


Sat Apr  5 06:25:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----